# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [35]:
with open("real_estate_sales_data.txt",encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [36]:
from langchain.text_splitter import CharacterTextSplitter

In [37]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [38]:
docs = text_splitter.create_documents([real_estate_sales])

In [39]:
docs[0]

Document(page_content='[客户问题]\n这款车的油耗高吗？\n[销售回答]\n这款车采用了最新的节能技术，市区油耗和高速油耗都非常经济，非常适合日常使用。')

In [40]:
len(docs)

10

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [41]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [42]:
query = "这款车的安全性能如何"

In [43]:
answer_list = db.similarity_search(query)

In [44]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题]
这款车的安全性能如何？
[销售回答]
这款车配备了多项主动和被动安全措施，包括多个安全气囊、ABS、ESC等，最大限度地保护您的安全。

[客户问题]
这款车适合家庭使用吗？
[销售回答]
这款车非常适合家庭使用，不仅空间大，而且安全配置齐全，能够很好地满足家庭出行的需求。

[客户问题]
这款车的座椅舒适吗？
[销售回答]
这款车的座椅设计非常符合人体工程学，材质柔软且支撑力好，长时间驾驶也不会感到疲劳。

[客户问题]
这款车的动力怎么样？
[销售回答]
这款车搭载了先进的发动机，动力强劲，无论是城市行驶还是长途驾驶，都能满足您的需求。



In [45]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [46]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [47]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000250C1680370>, search_kwargs={'k': 3})

In [48]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]
这款车的安全性能如何？
[销售回答]
这款车配备了多项主动和被动安全措施，包括多个安全气囊、ABS、ESC等，最大限度地保护您的安全。

[客户问题]
这款车适合家庭使用吗？
[销售回答]
这款车非常适合家庭使用，不仅空间大，而且安全配置齐全，能够很好地满足家庭出行的需求。

[客户问题]
这款车的座椅舒适吗？
[销售回答]
这款车的座椅设计非常符合人体工程学，材质柔软且支撑力好，长时间驾驶也不会感到疲劳。



In [49]:
docs = topK_retriever.get_relevant_documents("这款车的保值率怎么样?")

In [50]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]
这款车的保值率怎么样？
[销售回答]
我们品牌的车型在市场上的保值率一向很高，因为我们的车辆质量好，口碑佳，售后服务也很完善。

[客户问题]
这款车的售后服务怎么样？
[销售回答]
我们的售后服务非常完善，不仅提供免费保养、维修，还提供24小时道路救援服务，确保您无后顾之忧。

[客户问题]
这款车的动力怎么样？
[销售回答]
这款车搭载了先进的发动机，动力强劲，无论是城市行驶还是长途驾驶，都能满足您的需求。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [27]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [28]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]
这款车的安全性能如何？
[销售回答]
这款车配备了多项主动和被动安全措施，包括多个安全气囊、ABS、ESC等，最大限度地保护您的安全。

[客户问题]
这款车适合家庭使用吗？
[销售回答]
这款车非常适合家庭使用，不仅空间大，而且安全配置齐全，能够很好地满足家庭出行的需求。



### 提取向量数据库中的`销售回答`

In [29]:
docs = retriever.get_relevant_documents(query)

In [30]:
docs[0].page_content

'[客户问题]\n这款车的安全性能如何？\n[销售回答]\n这款车配备了多项主动和被动安全措施，包括多个安全气囊、ABS、ESC等，最大限度地保护您的安全。'

In [31]:
docs[0].page_content.split("[销售回答] ")

['[客户问题]\n这款车的安全性能如何？\n[销售回答]\n这款车配备了多项主动和被动安全措施，包括多个安全气囊、ABS、ESC等，最大限度地保护您的安全。']

In [32]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [33]:
ans

'[客户问题]\n这款车的安全性能如何？\n[销售回答]\n这款车配备了多项主动和被动安全措施，包括多个安全气囊、ABS、ESC等，最大限度地保护您的安全。'

#### 尝试各种问题

In [52]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [53]:
query = "我想座椅舒适的"

print(sales(query))

[]


C:\ProgramData\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [54]:
print(sales(query, 0.75))

['[客户问题]\n这款车的座椅舒适吗？\n[销售回答]\n这款车的座椅设计非常符合人体工程学，材质柔软且支撑力好，长时间驾驶也不会感到疲劳。']


In [55]:
query = "要座椅舒适的"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: ['[客户问题]\n这款车的座椅舒适吗？\n[销售回答]\n这款车的座椅设计非常符合人体工程学，材质柔软且支撑力好，长时间驾驶也不会感到疲劳。']

score:0.75 ans: ['[客户问题]\n这款车的座椅舒适吗？\n[销售回答]\n这款车的座椅设计非常符合人体工程学，材质柔软且支撑力好，长时间驾驶也不会感到疲劳。']

score:0.5 ans: ['[客户问题]\n这款车的座椅舒适吗？\n[销售回答]\n这款车的座椅设计非常符合人体工程学，材质柔软且支撑力好，长时间驾驶也不会感到疲劳。', '[客户问题]\n这款车的空间怎么样？\n[销售回答]\n这款车的内部空间非常宽敞，无论是前排还是后排，腿部和头部空间都很充裕，后备箱的储物空间也非常大。', '[客户问题]\n这款车适合家庭使用吗？\n[销售回答]\n这款车非常适合家庭使用，不仅空间大，而且安全配置齐全，能够很好地满足家庭出行的需求。', '[客户问题]\n这款车的配置有哪些？\n[销售回答]\n这款车配置非常丰富，包括全景天窗、智能导航系统、自动空调等，让您的驾乘体验更加舒适和便捷。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [29]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [31]:
qa_chain({"query": "你们小区有200万的房子吗？"})

{'query': '你们小区有200万的房子吗？',
 'result': '对不起，我无法回答这个问题，因为我是一个人工智能，没有实时的房地产信息。建议你直接联系房地产经纪人或者查阅相关房地产网站获取信息。'}

In [32]:
qa_chain({"query": "小区吵不吵"})

{'query': '小区吵不吵',
 'result': '这个小区特别注重居住体验，有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。所以，小区应该不会太吵。'}

In [33]:
print(sales("小区吵不吵"))

['这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']


## 加载 FAISS 向量数据库已有结果

In [34]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings())

In [35]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [36]:
qa_chain({"query": "我想买别墅，你们有么"})

{'query': '我想买别墅，你们有么',
 'result': '对不起，我无法提供这样的服务。我是一个人工智能，我可以帮助回答问题，但我不能出售商品或房产。'}

In [37]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [38]:
qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能帮助你购买别墅，因为我是一个人工智能，没有提供房地产服务的功能。'}

In [39]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [40]:
result = qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [41]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮你购买别墅。我是一个AI助手，我主要用来提供信息和回答问题。',
 'source_documents': []}